# Ejercicio 2 - Nominal Change Rate

In [2]:
import pandas as pd
import numpy as np

# Lectura y exploración de datos

In [3]:
#exploro los datos
df= pd.read_excel("/content/nominalmonthlycountryexchangerates_1_.xls") 
df.head(15)



,Nominal monthly average exchange rates (local currency per $US),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79
0,Updated 4/3/2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Source: International Financial Statistics of ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Contact: Kari Heerman (202-694-5250, keheerman...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Note: cells in blue are analyst projections.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Region,North America,NaN,Central America,NaN,NaN,NaN,NaN,NaN,Caribbean,NaN,NaN,South America,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Europe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Middle East,NaN,NaN,NaN,NaN,South Asia,NaN,NaN,NaN,Southeast Asia,NaN,NaN,NaN,NaN,Northeast Asia,NaN,NaN,NaN,NaN,Oceania,NaN,North Africa,NaN,NaN,NaN,Sub-Shaharan Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Date,Canada,Mexico,Guatemala,El Salvador,Honduras,Nicaragua,Costa Rica,Panama,Jamaica,Dominican Republic,Trinadad & Tobago,Colombia,Venezuela,Ecuador,Peru,Chile,Brazil,Paraguay,Uruguay,Argentina,Sweden,Norway,Finland,Denmark,U.K.,Ireland,Belgium & Luxembourg,Netherlands,France,Germany,Austria,Czech Republic,Hungary,Switzerland,Poland,Russia,Spain,Portugal,Italy,Greece,Turkey,Syria,Israel,Jordan,Kuwait,Saudi Arabia,India,Pakistan,Bangladesh,Sri Lanka,Thailand,Malaysia,Singapore,

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 563 entries, 0 to 562
Data columns (total 80 columns):
 #   Column                                                           Non-Null Count  Dtype 
---  ------                                                           --------------  ----- 
 0   Nominal monthly average exchange rates (local currency per $US)  560 non-null    object
 1   Unnamed: 1                                                       556 non-null    object
 2   Unnamed: 2                                                       555 non-null    object
 3   Unnamed: 3                                                       556 non-null    object
 4   Unnamed: 4                                                       555 non-null    object
 5   Unnamed: 5                                                       555 non-null    object
 6   Unnamed: 6                                                       555 non-null    object
 7   Unnamed: 7                                           

# Limpieza de datos

## Tabla países

In [5]:
#cambiamos el nombre de la primera columna por comodidad a la hora de limpiar los datos
#después filtramos las filas que nos interesan
df1=df\
.rename(columns={"Nominal monthly average exchange rates (local currency per $US)": "New_name"})\
.query("New_name.isin(['Region', 'Date', 'Country code', 'Month/currency'])", engine= "python")

df1

,New_name,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79
6,Region,North America,NaN,Central America,NaN,NaN,NaN,NaN,NaN,Caribbean,NaN,NaN,South America,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Europe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Middle East,NaN,NaN,NaN,NaN,South Asia,NaN,NaN,NaN,Southeast Asia,NaN,NaN,NaN,NaN,Northeast Asia,NaN,NaN,NaN,NaN,Oceania,NaN,North Africa,NaN,NaN,NaN,Sub-Shaharan Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Date,Canada,Mexico,Guatemala,El Salvador,Honduras,Nicaragua,Costa Rica,Panama,Jamaica,Dominican Republic,Trinadad & Tobago,Colombia,Venezuela,Ecuador,Peru,Chile,Brazil,Paraguay,Uruguay,Argentina,Sweden,Norway,Finland,Denmark,U.K.,Ireland,Belgium & Luxembourg,Netherlands,France,Germany,Austria,Czech Republic,Hungary,Switzerland,Poland,Russia,Spain,Portugal,Italy,Greece,Turkey,Syria,Israel,Jordan,Kuwait,Saudi Arabia,India,Pakistan,Bangladesh,Sri Lanka,Thailand,Malaysia,Singapore,Indonesia,Philippines,"China,PR",South Korea,Hong Kong,Taiwan,Japan,Australia,New Zealand,Morocco,Algeria,Tunisia,Egypt,Cameroon,Senegal,Sierra Leone,Cote d'Ivoire,Ghana,Nigeria,Benin,Congo,Kenya,Tanzania,Mozambique,South Africa,Zambia
8,Country code,(1220),(2010),(2050),(2110),(2150),(2190),(2230),(2250),(2410),(2470),(2740),(3010),(3070),(3310),(3330),(3370),(3510),(3530),(3550),(3570),(4010),(4039),(4050),(4099),(4120),(4190),(0303),(4210),(4279),(0305),(4330),(4351),(4370),(4419),(4550),(4621),(4700),(4710),(4759),(4840),(4890),(5020),(5081),(5110),(5130),(5170),(5330),(5350),(5380),(5420),(5490),(5570),(5590),(5600),(5650,(5700),(5800),(5820),(5830),(5880),(6021),(6141),(7140),(7210),(7230),(7290),(7420),(7440),(7470),(7480),(7490),(7530),(7610),(7660),(7790),(7830),(7870),(7910),(7940)
9,Month/currency,C$,Pesos,Quetzales,Colones,Lempiras,Gold cordobas,Colones,Balboas,J$,Pesos,TT$,Pesos,Bolivares,Sucres,New soles,Pesos,Nominals,Guaranies,Pesos,Pesos,Kronor,Kroner,Euro,Kroner,Pounds,Euro,Euro,Euro,Euro,Euro,Euro,Koruny,Forint,Francs,Zlotys,Rubles,Euro,Euro,Euro,Euro,Liras,Pounds,Nsheqalim,Dinars,Dinars,Riyals,Rupees,Rupees,Taka,Rupees,Baht,Ringgit,S$,Rupiah,Pesos,Yuan,Won,HK$,T$,Yen,A$,NZ$,Dirhams,Dinars,Dinars,Pounds,Francs,Francs,Leones,Francs,Cedis,Naira,Francs,Francs,Shillings,Shillings,Meticais,Rand,Kwacha


In [6]:
#Utilizo el método transpose para transponer la tabla, colocando el nombre de las columnas iniciales 
#como índice
df1=df1.transpose()
df1




,6,7,8,9
New_name,Region,Date,Country code,Month/currency
Unnamed: 1,North America,Canada,(1220),C$
Unnamed: 2,NaN,Mexico,(2010),Pesos
Unnamed: 3,Central America,Guatemala,(2050),Quetzales
Unnamed: 4,NaN,El Salvador,(2110),Colones
...,...,...,...,...
Unnamed: 75,NaN,Kenya,(7790),Shillings
Unnamed: 76,NaN,Tanzania,(7830),Shillings
Unnamed: 77,NaN,Mozambique,(7870),Meticais
Unnamed: 78,NaN,South Africa,(7910),Rand


In [7]:
#Renombro las columnas de manera que tomen el nombre de los valores de la primera fila
df1=df1.rename(columns=df1.iloc[0]).drop(df1.index[0])
df1

,Region,Date,Country code,Month/currency
Unnamed: 1,North America,Canada,(1220),C$
Unnamed: 2,NaN,Mexico,(2010),Pesos
Unnamed: 3,Central America,Guatemala,(2050),Quetzales
Unnamed: 4,NaN,El Salvador,(2110),Colones
Unnamed: 5,NaN,Honduras,(2150),Lempiras
...,...,...,...,...
Unnamed: 75,NaN,Kenya,(7790),Shillings
Unnamed: 76,NaN,Tanzania,(7830),Shillings
Unnamed: 77,NaN,Mozambique,(7870),Meticais
Unnamed: 78,NaN,South Africa,(7910),Rand


In [8]:
#Reseteamos el índice 
#Quitamos los valores nulos, pero lo hacemos con fillna porque el archivo es un excel y observando
#vemos que se trata del mismo valor dentro de la columna Region porque viene de las merged cells de excel
df1=df1\
.reset_index(drop=True)\
.assign(Region = lambda dataset: dataset.Region.fillna(method = 'ffill'))
df1

,Region,Date,Country code,Month/currency
0,North America,Canada,(1220),C$
1,North America,Mexico,(2010),Pesos
2,Central America,Guatemala,(2050),Quetzales
3,Central America,El Salvador,(2110),Colones
4,Central America,Honduras,(2150),Lempiras
...,...,...,...,...
74,Sub-Shaharan Africa,Kenya,(7790),Shillings
75,Sub-Shaharan Africa,Tanzania,(7830),Shillings
76,Sub-Shaharan Africa,Mozambique,(7870),Meticais
77,Sub-Shaharan Africa,South Africa,(7910),Rand


In [9]:
#Quitamos los parentesis de los valores de la columna Country code, para ello creo otra columna igual
#pero con los valores sin paréntesis

import regex as re
numeros= r'(\d+)'
def formateo(x):
    x=re.search(numeros,x).group()
    return x


df1=df1.assign(
    CountryCode = lambda dataset: dataset["Country code"].map(lambda value: formateo(value))
)

df1



,Region,Date,Country code,Month/currency,CountryCode
0,North America,Canada,(1220),C$,1220
1,North America,Mexico,(2010),Pesos,2010
2,Central America,Guatemala,(2050),Quetzales,2050
3,Central America,El Salvador,(2110),Colones,2110
4,Central America,Honduras,(2150),Lempiras,2150
...,...,...,...,...,...
74,Sub-Shaharan Africa,Kenya,(7790),Shillings,7790
75,Sub-Shaharan Africa,Tanzania,(7830),Shillings,7830
76,Sub-Shaharan Africa,Mozambique,(7870),Meticais,7870
77,Sub-Shaharan Africa,South Africa,(7910),Rand,7910


In [10]:
#elimino la columna Country code con los valores que tenían el paréntesis
#le pongo bien el nombre a la columna Date y Month/currency que realmente su nombre debe ser Country
#ordeno las columnas

df_country=df1\
.drop(columns=["Country code"])\
.rename(columns={"Date":"Country", "Month/currency":"Currency"})\
.filter(["CountryCode", "Country", "Region", "Currency"])
df_country

,CountryCode,Country,Region,Currency
0,1220,Canada,North America,C$
1,2010,Mexico,North America,Pesos
2,2050,Guatemala,Central America,Quetzales
3,2110,El Salvador,Central America,Colones
4,2150,Honduras,Central America,Lempiras
...,...,...,...,...
74,7790,Kenya,Sub-Shaharan Africa,Shillings
75,7830,Tanzania,Sub-Shaharan Africa,Shillings
76,7870,Mozambique,Sub-Shaharan Africa,Meticais
77,7910,South Africa,Sub-Shaharan Africa,Rand


In [11]:
#verificamos los tipos de la tabla country
df_country.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   CountryCode  79 non-null     object
 1   Country      79 non-null     object
 2   Region       79 non-null     object
 3   Currency     79 non-null     object
dtypes: object(4)
memory usage: 2.6+ KB


## Tabla monedas

In [12]:
df.head(20)

,Nominal monthly average exchange rates (local currency per $US),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79
0,Updated 4/3/2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Source: International Financial Statistics of ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Contact: Kari Heerman (202-694-5250, keheerman...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Note: cells in blue are analyst projections.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Region,North America,NaN,Central America,NaN,NaN,NaN,NaN,NaN,Caribbean,NaN,NaN,South America,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Europe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Middle East,NaN,NaN,NaN,NaN,South Asia,NaN,NaN,NaN,Southeast Asia,NaN,NaN,NaN,NaN,Northeast Asia,NaN,NaN,NaN,NaN,Oceania,NaN,North Africa,NaN,NaN,NaN,Sub-Shaharan Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Date,Canada,Mexico,Guatemala,El Salvador,Honduras,Nicaragua,Costa Rica,Panama,Jamaica,Dominican Republic,Trinadad & Tobago,Colombia,Venezuela,Ecuador,Peru,Chile,Brazil,Paraguay,Uruguay,Argentina,Sweden,Norway,Finland,Denmark,U.K.,Ireland,Belgium & Luxembourg,Netherlands,France,Germany,Austria,Czech Republic,Hungary,Switzerland,Poland,Russia,Spain,Portugal,Italy,Greece,Turkey,Syria,Israel,Jordan,Kuwait,Saudi Arabia,India,Pakistan,Bangladesh,Sri Lanka,Thailand,Malaysia,Singapore,

In [13]:
#Cambiamos la cabecera, renombrando las columnas de manera que tomen el nombre de la fila Country code
#Reseteamos el índice
df=df\
.rename(columns=df.iloc[8]).drop(df1.index[8])\
.reset_index(drop=True)
df.head(10)


,Country code,(1220),(2010),(2050),(2110),(2150),(2190),(2230),(2250),(2410),(2470),(2740),(3010),(3070),(3310),(3330),(3370),(3510),(3530),(3550),(3570),(4010),(4039),(4050),(4099),(4120),(4190),(0303),(4210),(4279),(0305),(4330),(4351),(4370),(4419),(4550),(4621),(4700),(4710),(4759),(4840),(4890),(5020),(5081),(5110),(5130),(5170),(5330),(5350),(5380),(5420),(5490),(5570),(5590),(5600),(5650,(5700),(5800),(5820),(5830),(5880),(6021),(6141),(7140),(7210),(7230),(7290),(7420),(7440),(7470),(7480),(7490),(7530),(7610),(7660),(7790),(7830),(7870),(7910),(7940)
0,Updated 4/3/2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Source: International Financial Statistics of ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Contact: Kari Heerman (202-694-5250, keheerman...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Note: cells in blue are analyst projections.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Region,North America,NaN,Central America,NaN,NaN,NaN,NaN,NaN,Caribbean,NaN,NaN,South America,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Europe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Middle East,NaN,NaN,NaN,NaN,South Asia,NaN,NaN,NaN,Southeast Asia,NaN,NaN,NaN,NaN,Northeast Asia,NaN,NaN,NaN,NaN,Oceania,NaN,North Africa,NaN,NaN,NaN,Sub-Shaharan Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Date,Canada,Mexico,Guatemala,El Salvador,Honduras,Nicaragua,Costa Rica,Panama,Jamaica,Dominican Republic,Trinadad & Tobago,Colombia,Venezuela,Ecuador,Peru,Chile,Brazil,Paraguay,Uruguay,Argentina,Sweden,Norway,Finland,Denmark,U.K.,Ireland,Belgium & Luxembourg,Netherlands,France,Germany,Austria,Czech Republic,Hungary,Switzerland,Poland,Russia,Spain,Portugal,Italy,Greece,Turkey,Syria,Israel,Jordan,Kuwait,Saudi Arabia,India,Pakistan,Bangladesh,Sri Lanka,Thailand,Malaysia,Singapore,Indonesia,Philippines,"China,PR",South Korea,Hong Kong,Taiwan,Japan,Australia,New Zealand,Morocco,Algeria,Tunisia,Egypt,Cameroon,Senegal,Sierra Leone,Cote d'Ivoire,Ghana,Nigeria,Benin,Congo,Kenya,Tanzania,Mozambique,South Africa,Zambia
8,Month/currency,C$,Pesos,Quetzales,Colones,Lempiras,Gold cordobas,Colones,Balboas,J$,Pesos,TT$,Pesos,Bolivares,Sucres,New soles,Pesos,Nominals,Guaranies,Pesos,Pesos,Kronor,Kroner,Euro,Kroner,Pounds,Eur

In [14]:
#eliminamos valores nan de la columna
df= df\
.dropna(how="all")\
.reset_index(drop=True)

df.head(20)

,Country code,(1220),(2010),(2050),(2110),(2150),(2190),(2230),(2250),(2410),(2470),(2740),(3010),(3070),(3310),(3330),(3370),(3510),(3530),(3550),(3570),(4010),(4039),(4050),(4099),(4120),(4190),(0303),(4210),(4279),(0305),(4330),(4351),(4370),(4419),(4550),(4621),(4700),(4710),(4759),(4840),(4890),(5020),(5081),(5110),(5130),(5170),(5330),(5350),(5380),(5420),(5490),(5570),(5590),(5600),(5650,(5700),(5800),(5820),(5830),(5880),(6021),(6141),(7140),(7210),(7230),(7290),(7420),(7440),(7470),(7480),(7490),(7530),(7610),(7660),(7790),(7830),(7870),(7910),(7940)
0,Updated 4/3/2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Source: International Financial Statistics of ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Contact: Kari Heerman (202-694-5250, keheerman...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Note: cells in blue are analyst projections.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Region,North America,NaN,Central America,NaN,NaN,NaN,NaN,NaN,Caribbean,NaN,NaN,South America,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Europe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Middle East,NaN,NaN,NaN,NaN,South Asia,NaN,NaN,NaN,Southeast Asia,NaN,NaN,NaN,NaN,Northeast Asia,NaN,NaN,NaN,NaN,Oceania,NaN,North Africa,NaN,NaN,NaN,Sub-Shaharan Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Date,Canada,Mexico,Guatemala,El Salvador,Honduras,Nicaragua,Costa Rica,Panama,Jamaica,Dominican Republic,Trinadad & Tobago,Colombia,Venezuela,Ecuador,Peru,Chile,Brazil,Paraguay,Uruguay,Argentina,Sweden,Norway,Finland,Denmark,U.K.,Ireland,Belgium & Luxembourg,Netherlands,France,Germany,Austria,Czech Republic,Hungary,Switzerland,Poland,Russia,Spain,Portugal,Italy,Greece,Turkey,Syria,Israel,Jordan,Kuwait,Saudi Arabia,India,Pakistan,Bangladesh,Sri Lanka,Thailand,Malaysia,Singapore,Indonesia,Philippines,"China,PR",South Korea,Hong Kong,Taiwan,Japan,Australia,New Zealand,Morocco,Algeria,Tunisia,Egypt,Cameroon,Senegal,Sierra Leone,Cote d'Ivoire,Ghana,Nigeria,Benin,Congo,Kenya,Tanzania,Mozambique,South Africa,Zambia
6,Month/currency,C$,Pesos,Quetzales,Colones,Lempiras,Gold cordobas,Colones,Balboas,J$,Pesos,TT$,Pesos,Bolivares,Sucres,New soles,Pesos,Nominals,Guaranies,Pesos,Pesos,Kronor,Kroner,Euro,Kroner,Pounds,Euro,Euro,Euro,Euro,Euro,Euro,Koruny,Forint,Francs,Zlotys,Rubles,Euro,Euro,Euro,Euro,Liras,Pounds,Nsheqalim,Dinars,Dinars,Riyals,Rupees,Rupees,Taka,Rupees,Baht,Ringgit,S$,Rupiah,Pesos,Yuan,Won,HK$,T$,Yen,A$,NZ$,Dirhams,Dinars,Dinars,Pounds,Francs,Francs,Leones,Francs,Cedis,Naira,Francs,Francs,Shillings,Shillings,Meticais,Rand,Kwacha
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"China,PR",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [15]:
#Dejamos solo las filas que tienen fechas dentro de la columna Date
#renombramos la columna Country code por Date
df= df\
.rename(columns={"Country code":"Date"})\
.iloc[8: ]
df.head(10)


,Date,(1220),(2010),(2050),(2110),(2150),(2190),(2230),(2250),(2410),(2470),(2740),(3010),(3070),(3310),(3330),(3370),(3510),(3530),(3550),(3570),(4010),(4039),(4050),(4099),(4120),(4190),(0303),(4210),(4279),(0305),(4330),(4351),(4370),(4419),(4550),(4621),(4700),(4710),(4759),(4840),(4890),(5020),(5081),(5110),(5130),(5170),(5330),(5350),(5380),(5420),(5490),(5570),(5590),(5600),(5650,(5700),(5800),(5820),(5830),(5880),(6021),(6141),(7140),(7210),(7230),(7290),(7420),(7440),(7470),(7480),(7490),(7530),(7610),(7660),(7790),(7830),(7870),(7910),(7940)
8,1970-01-01 00:00:00,1.0728,0.0125,1,2.5,2,0,6.635,1,0.8333,1,1.9996,17.9,0.00445,18,1.24592e-07,0.01,1.58182e-12,126,0.0002,3.51548e-11,5.1666,7.1515,0.705717,7.4968,0.41658,0.528969,1.23947,1.64881,0.846565,1.88478,1.88034,21.1173,60,4.299,0.0004,0.000364263,0.42021,0.142342,0.325022,0.0880411,9e-06,3.82,0.0003,0.357143,0.357143,4.5,7.553,4.7877,7.5472,5.958,20.9281,3.0764,3.09,326,3.94,2.462,304.85,6.0678,38,357.73,0.89622,0.89326,5.0595,4.9371,0.52,0.43478,277.66,277.66,0.833,277.66,0.000102,0.7143,277.66,277.66,7.1429,7.143,0.0514539,0.71512,0.00071
9,1970-02-01 00:00:00,1.0731,0.0125,1,2.5,2,0,6.635,1,0.8333,1,1.9961,18,0.00445,18,1.25146e-07,0.01,1.60109e-12,126,0.0002,3.51548e-11,5.18,7.148,0.704707,7.4979,0.415852,0.528023,1.23947,1.65094,0.844857,1.88595,1.87968,21.125,60,4.309,0.0004,0.000364263,0.419927,0.142342,0.325104,0.0880411,9e-06,3.82,0.0003,0.357143,0.357143,4.5,7.541,4.7724,7.5472,5.958,20.9281,3.073,3.09,326,4.2043,2.462,305.86,6.0572,38,357.65,0.89469,0.89174,5.0512,4.9371,0.52,0.43478,277.2,277.2,0.832,277.2,0.000102,0.7143,277.2,277.2,7.1429,7.143,0.0514539,0.71386,0.00071
10,1970-03-01 00:00:00,1.0727,0.0125,1,2.5,2,0,6.635,1,0.8333,1,1.9951,18.09,0.00445,18,1.25711e-07,0.01,1.60727e-12,126,0.0002,3.51548e-11,5.1997,7.1424,0.703698,7.4963,0.415645,0.527782,1.23947,1.64863,0.84504,1.87859,1.87968,21.1327,60,4.302,0.0004,0.000365112,0.419573,0.142412,0.324877,0.0880411,9e-06,3.82,0.0003,0.357143,0.357143,4.5,7.536,4.7775,7.5472,5.958,20.9281,3.0715,3.09,326,6.0962,2.462,306.63,6.0542,38,357.52,0.89421,0.89119,5.0504,4.9371,0.52,0.43478,277.16,277.16,0.831,277.16,0.000102,0.7143,277.16,277.16,7.1429,7.143,0.0516344,0.71347,0.00071
11,1970-04-01 00:00:00,1.0728,0.0125,1,2.5,2,0,6.635,1,0.8333,1,1.9948,18.2,0.00445,18,1.26286e-07,0.011,1.63273e-12,126,0.0002,3.51548e-11,5.1994,7.1424,0.702521,7.5047,0.415576,0.527716,1.23947,1.64813,0.843287,1.86218,1.88026,21.1404,60,4.31,0.0004,0.000366025,0.419362,0.142467,0.324919,0.0880411,9e-06,3.82,0.0003,0.357143,0.357143,4.5,7.536,4.7766,7.5472,5.958,20.9281,3.0697,3.09,352,6.1628,2.462,307.73,6.0531,38,357.99,0.89413,0.89111,5.0399,4.9371,0.52,0.43478,276.58,276.58,0.831,276.58,0.000102,0.7143,276.58,276.58,7.1429,7.143,0.0516344,0.71336,0.00071
12,1970-05-01 00:00:00,1.0728,0.0125,1,2.5,2,0,6.635,1,0.8333,1,1.9967,18.28,0.00445,18,1.26873e-07,0.011,1.64364e-12,126,0.0002,3.51548e-11,5.1994,7.1495,0.702353,7.5053,0.415973,0.528243,1.23947,1.64622,0.841717,1.85763,1.88201,21.1481,60,4.314,0.0004,0.00036694,0.418875,0.142601,0.324877,0.0880411,9e-06,3.82,0.0003,0.357143,0.357143,4.5,7.541,4.7809,7.5472,5.958,20.9281,3.0731,3.09,378,6.1213,2.462,308.76,6.0589,38,358.89,0.89502,0.8919,5.0305,4.9371,0.52,0.43478,276.07,276.07,0.832,276.07,0.000102,0.7143,276.07,276.07,7.1429,7.143,0.0516344,0.71403,0.00071
13,1970-06-01 00:00:00,1.0384,0.0125,1,2.5,2,0,6.635,1,0.8333,1,2.0018,18.38,0.00445,18,1.27472e-07,0.011,1.65818e-12,126,0.0002,3.81681e-11,5.1905,7.1505,0.703194,7.4996,0.417049,0.529565,1.23947,1.64486,0.841595,1.85722,1.88186,21.1558,60,4.303,0.0004,0.000367858,0.418527,0.142841,0.324877,0.0880411,9e-06,3.82,0.0003,0.357143,0.357143,4.5,7.555,4.7925,7.54519,5.958,20.9281,3.0806,3.095,378,6.1924,2.462,309.89,6.0746,38,358.76,0.89726,0.89413,5.0298,4.9371,0.52,0.43478,276.03,276.03,0.834,276.03,0.000102,0.7143,276.03,276.03,7.1429,7.143,0.0516344,0.7158,0.00071
14,1970-07-01 00:00:00,1.032,0.0125,1,2.5,2,0,6.635,1,0

In [16]:
#Estrechamos la tabla 

df=df.melt(id_vars= "Date", var_name= "CountryCode", value_name= "Change")
df



,Date,CountryCode,Change
0,1970-01-01,(1220),1.0728
1,1970-02-01,(1220),1.0731
2,1970-03-01,(1220),1.0727
3,1970-04-01,(1220),1.0728
4,1970-05-01,(1220),1.0728
...,...,...,...
43603,2015-08-01,(7940),7.2616
43604,2015-09-01,(7940),7.25099
43605,2015-10-01,(7940),7.24037
43606,2015-11-01,(7940),7.22975


In [17]:
#quitamos los paréntesis de la columna CountryCode y aprovechamos la misma función que utilizamos para
#la otra tabla
df_monedas=df.assign(
    CountryCode = lambda dataset: dataset["CountryCode"].map(lambda value: formateo(value))
)
df_monedas

,Date,CountryCode,Change
0,1970-01-01,1220,1.0728
1,1970-02-01,1220,1.0731
2,1970-03-01,1220,1.0727
3,1970-04-01,1220,1.0728
4,1970-05-01,1220,1.0728
...,...,...,...
43603,2015-08-01,7940,7.2616
43604,2015-09-01,7940,7.25099
43605,2015-10-01,7940,7.24037
43606,2015-11-01,7940,7.22975


In [18]:
#comprobamos que los datatype son los correctos y vemos que el change es object y debería de ser float64
df_monedas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43608 entries, 0 to 43607
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         43608 non-null  datetime64[ns]
 1   CountryCode  43608 non-null  object        
 2   Change       43608 non-null  object        
dtypes: datetime64[ns](1), object(2)
memory usage: 1022.2+ KB


In [19]:
df_monedas=df_monedas.assign(Change= lambda dataset: dataset.Change.astype("float"))
df_monedas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43608 entries, 0 to 43607
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         43608 non-null  datetime64[ns]
 1   CountryCode  43608 non-null  object        
 2   Change       43608 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 1022.2+ KB


In [20]:
#Unimos la tabla países y tabla monedas

df_final= df_country.join(df_monedas.set_index('CountryCode'), on='CountryCode')



# Verificación de datos

In [21]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43608 entries, 0 to 78
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   CountryCode  43608 non-null  object        
 1   Country      43608 non-null  object        
 2   Region       43608 non-null  object        
 3   Currency     43608 non-null  object        
 4   Date         43608 non-null  datetime64[ns]
 5   Change       43608 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 2.3+ MB


In [22]:
df_final.sample(10)

,CountryCode,Country,Region,Currency,Date,Change
76,7870,Mozambique,Sub-Shaharan Africa,Meticais,1989-02-01,0.700000
9,2470,Dominican Republic,Caribbean,Pesos,2012-11-01,40.074400
6,2230,Costa Rica,Central America,Colones,1971-04-01,6.635000
28,4279,France,Europe,Euro,2007-07-01,0.729120
65,7290,Egypt,North Africa,Pounds,1994-02-01,3.376000
40,4890,Turkey,Europe,Liras,1990-04-01,0.002487
62,7140,Morocco,North Africa,Dirhams,1994-02-01,9.643600
22,4050,Finland,Europe,Euro,1975-12-01,0.651392
21,4039,Norway,Europe,Kroner,1999-01-01,7.452400
4,2150,Honduras,Central America,Lempiras,1982-02-01,2.000000


In [23]:
df_final.describe(include= 'all', datetime_is_numeric= True)

,CountryCode,Country,Region,Currency,Date,Change
count,43608,43608,43608,43608,43608,43608.000000
unique,79,79,12,50,NaN,NaN
top,6021,Jamaica,Europe,Euro,NaN,NaN
freq,552,552,11592,6072,NaN,NaN
mean,NaN,NaN,NaN,NaN,1992-12-15 16:46:57.391314688,297.224336
min,NaN,NaN,NaN,NaN,1970-01-01 00:00:00,0.000000
25%,NaN,NaN,NaN,NaN,1981-06-23 12:00:00,0.896972
50%,NaN,NaN,NaN,NaN,1992-12-16 12:00:00,4.202000
75%,NaN,NaN,NaN,NaN,2004-06-08 12:00:00,33.150000
max,NaN,NaN,NaN,NaN,2015-12-01 00:00:00,25000.000000


In [24]:
df_final.query(
    "Country == 'Spain' \
    and Date.between( '1974-05-01', '1975-01-12')", engine = "python")

,CountryCode,Country,Region,Currency,Date,Change
36,4700,Spain,Europe,Euro,1974-05-01,0.345083
36,4700,Spain,Europe,Euro,1974-06-01,0.344224
36,4700,Spain,Europe,Euro,1974-07-01,0.342841
36,4700,Spain,Europe,Euro,1974-08-01,0.344025
36,4700,Spain,Europe,Euro,1974-09-01,0.346610
36,4700,Spain,Europe,Euro,1974-10-01,0.345071
36,4700,Spain,Europe,Euro,1974-11-01,0.343034
36,4700,Spain,Europe,Euro,1974-12-01,0.339440
36,4700,Spain,Europe,Euro,1975-01-01,0.337745


In [25]:
pd.set_option('display.max_rows', 12)
df_final.groupby("Region").agg(
    {"Change": "mean", "Country": "count", "Currency": "last"})

,Change,Country,Currency
Region,,,
Caribbean,17.762350,1656,TT$
Central America,40.818580,3312,Balboas
Europe,9.428209,11592,Liras
Middle East,3.031537,2760,Riyals
North Africa,12.144625,2208,Pounds
North America,3.330150,1104,Pesos
Northeast Asia,211.053600,2760,Yen
Oceania,1.323456,1104,NZ$
South America,1453.344677,4968,Pesos
